<a href="https://colab.research.google.com/github/solobala/RMSL_9/blob/main/RS_lecture_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=879170 sha256=01fe1dbb957b16c429ed5dbcdffdb3acef004cb4d1f67d2e6dfd59ffbe04c165
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [23]:
import numpy as np

from lightfm.datasets import fetch_movielens # встроенный датасет

# movielens = fetch_movielens()
movielens = fetch_movielens(min_rating=5.0)

In [5]:
 # посмотреть параметры
?fetch_movielens

In [6]:
# Смотрим типы данных 2 матрицы train и testЮ фичи для фильмов
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse._coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse._csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [7]:
train = movielens['train']
test = movielens['test']

In [12]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k # считает по k первым элементам
from lightfm.evaluation import auc_score # Площади под кривой чтобы найти оптимальный порог предсказания - аналог roc_auc

model = LightFM(random_state=42)
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

In [13]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.44, test 0.13.


In [15]:
model = LightFM(learning_rate=0.05, loss='warp', random_state=42) # 'warp' - Алгоритм минимизации функции потерь

model.fit_partial(train, epochs=10) # запускаем для оптимизации не fit, а fit_partial - продолжает обучение с момента остановки

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.61, test 0.22.


In [25]:
def sample_recommendation(model, data, user_ids):


    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)



In [24]:
known_positives = movielens['item_labels'][movielens['train'].tocsr()[2].indices]

In [26]:
sample_recommendation(model, movielens, [3, 25, 450])

User 3
     Known positives:
        C
        A
        I
     Recommended:
        S
        A
        E
User 25
     Known positives:
        F
        G
        L
     Recommended:
        C
        S
        L
User 450
     Known positives:
        E
        S
        C
     Recommended:
        G
        D
        C
